In [ ]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(words) for words in sentence_words]
    return sentence_words

In [ ]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [ ]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})

    return return_list

In [ ]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
        
    return result

In [ ]:
print('GO! Bot is Running!')

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)
    if message == "goodbye":
        break

GO! Bot is Running!
Hello
Goodbye!, take care
Hi there
Hi
Who are you
I'm Neon, the customer service assistant of the company
What type of products does your company sell
We sell various types of products like electical appliances, mobile phones, T.V.,cosmetics, sports equipments, clothes, video games and many other kind of products.
What are your timings
You can buy at anytime you like, we all will be there to help you
oh great
Sad to see you leave
I got the wrong product
We're very sorry, next time we'll take care of that
goodbye
Goodbye!, take care
